En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
import os
import cProfile
import pstats
from memory_profiler import profile
from zipfile import ZipFile

file_path = "farmers-protest-tweets-2021-2-4.json"
data_folder_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))

# checkea si existe la carpeta data, sino, la crea
if not os.path.exists(data_folder_path):
    os.makedirs(data_folder_path)

# checkea si existe el archivo tweets.json.zip, si no, checkea si existe el archivo json, si no, lanza excepción
if not os.path.exists(f"{data_folder_path}\\tweets.json.zip"):
    if not os.path.exists(f"{data_folder_path}\\{file_path}"):
        raise FileNotFoundError("Por favor descargue el archivo 'tweets.json.zip' y colóquelo en la carpeta 'data'")

# si no existe el archivo json, descomprime el archivo zip
if not os.path.exists(f"{data_folder_path}\\{file_path}"):
    with ZipFile(f"{data_folder_path}\\tweets.json.zip", 'r') as zObject: 
        zObject.extractall(path=f"{data_folder_path}")

In [2]:
from q1_memory import q1_memory
q1_mem_result = q1_memory(f"{data_folder_path}/{file_path}")

Filename: c:\Users\BYB\Documents\Python\jiji\latam-de-challenge\src\q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    40     65.7 MiB     65.7 MiB           1   @profile
    41                                         def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    42                                             # Inicializacion de Spark
    43     68.4 MiB      2.8 MiB           1       spark = SparkClass("Q1: Memory")
    44                                             # Carga de datos
    45     68.6 MiB      0.1 MiB           1       json_data = spark.load_json(file_path)
    46                                             
    47     68.6 MiB      0.0 MiB           1       df = extract_all_tweets(json_data)
    48                                         
    49                                             # obtengo las top 10 fechas con mas tweets
    50     68.6 MiB      0.0 MiB           4       top_10_dates = df.groupBy(df["date"].

In [3]:
from q1_time import q1_time

with cProfile.Profile() as pr:
    q1_time(f"{data_folder_path}/{file_path}")

time_result = pstats.Stats(pr).strip_dirs().sort_stats('time')
time_result.print_stats()

         37316 function calls (36355 primitive calls) in 3.691 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      420    3.435    0.008    3.435    0.008 {method 'recv_into' of '_socket.socket' objects}
        8    0.174    0.022    0.174    0.022 {method 'acquire' of '_thread.lock' objects}
        2    0.017    0.009    0.017    0.009 {method 'connect' of '_socket.socket' objects}
      410    0.015    0.000    0.015    0.000 {method 'sendall' of '_socket.socket' objects}
      410    0.004    0.000    3.457    0.008 clientserver.py:499(send_command)
       47    0.003    0.000    0.003    0.000 {built-in method nt.stat}
     1161    0.002    0.000    0.002    0.000 protocol.py:214(smart_decode)
      411    0.002    0.000    3.435    0.008 {method 'readline' of '_io.BufferedReader' objects}
   359/11    0.001    0.000    0.003    0.000 types.py:1380(_parse_datatype_json_value)
      155    0.001    0.000    0.003    